In [1]:
# 이진화: 특정값을 기준으로 0 혹은 255로 나눈다. (흰색과 검은색으로)

import cv2
import numpy as np

book_path='images/book.jpg'
iu_path='images/iu.jpg'


In [ ]:
## 기존 이진화

img=cv2.imread(book_path, cv2.IMREAD_GRAYSCALE)    # 그레이스케일로 불러오기

ret , binary = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)    # 이미지, 임계값, 나누는 값중 최대값, 이진화를 계산하는 옵션

cv2.imshow('1', img)
cv2.imshow('2', binary)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 적응형 이진화: 이진화를 하는데, 글자부분(특정부분)이 살아나게 하고 싶다. 일부 블록만 다른 임계값을 적용한다.

img=cv2.imread(book_path, cv2.IMREAD_GRAYSCALE)    # 그레이스케일로 불러오기
name='Adaptive'
cv2.namedWindow(name)
cv2.createTrackbar('block_size',name, 25, 100, lambda x:x)
cv2.createTrackbar('C',name, 1, 10, lambda x:x) # 주로 10 안쪽으로 설정


while True:
    block_size=cv2.getTrackbarPos('block_size', name)
    C=cv2.getTrackbarPos('C', name)

    if block_size<=1:  # 블록사이즈는 1보다 큰 홀수 값이어야 하기 때문에 위와 같이 예외처리
        block_size=3

    if block_size%2==0:
        block_size+=1

    binary=cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, C)      # 이미지, 최대값, 적응형 이진화를 할 방법, 이진화 적용 방식(계산방법 옵션), 블록사이즈, C값(보정값=인계값에서 빼주는 값))
    cv2.imshow(name, binary)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

In [3]:
# 트랙바
## 직접 아핀 행렬 생성

import math

img=cv2.imread(iu_path)

# 트랙바
name='Trackbar'
cv2.namedWindow(name)
cv2.createTrackbar('rotation',name,0,360, lambda x:x)     # 트랙바 창 이름, 트랙바 조정 값 이름, 시작점, 최대점, 수식

while True:
    angle=cv2.getTrackbarPos('rotation',name)

    rad=angle*math.pi/180
    affine_matrix=np.array([[math.cos(rad),-math.sin(rad),0], [math.sin(rad),math.cos(rad),0]], dtype=np.float32)
    dst = cv2.warpAffine(img, affine_matrix, (img.shape[1], img.shape[0]))
    cv2.imshow(name, dst)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

In [15]:
## 기존 이진화 and 트랙바 실습

img=cv2.imread(book_path, cv2.IMREAD_GRAYSCALE)    # 그레이스케일로 불러오기
ret , binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)    # 이미지, 임계값, 나누는 값중 최대값, 이진화를 계산하는 옵션

# 트랙바
name='Trackbar'
cv2.namedWindow(name)
cv2.createTrackbar('Trackbar', name, 127, 255, lambda x:x)     # 트랙바 창 이름, 트랙바 조정 값 이름, 시작점, 최대점, 수식(무의미해도 없으면 안됨, 변동이 있을 시 변환)

while True:
    threshold=cv2.getTrackbarPos(name, 'Trackbar')
    ret , binary = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)    # 이미지, 임계값, 나누는 값중 최대값, 이진화를 계산하는 옵션

    cv2.imshow(name, binary)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

In [2]:
# 실습(컬러 팔레트)

img=np.zeros([640,400,3],dtype=np.uint8)

img[:]=(255,0,0)

name='Palette'
cv2.namedWindow(name)
cv2.createTrackbar('R', name, 0, 255, lambda x:x)     # 트랙바 창 이름, 트랙바 조정 값 이름, 시작점, 최대점, 수식(무의미해도 없으면 안됨, 변동이 있을 시 변환)
cv2.createTrackbar('G', name, 0, 255, lambda x:x)
cv2.createTrackbar('B', name, 0, 255, lambda x:x)
cv2.createTrackbar('0: OFF 1: ON', name, 1, 1, lambda x:x)

while True:
    threshold_1=cv2.getTrackbarPos('R', name)
    threshold_2=cv2.getTrackbarPos('G', name)
    threshold_3=cv2.getTrackbarPos('B', name)
    threshold_onoff=cv2.getTrackbarPos('0: OFF 1: ON', name)

    if threshold_onoff==1:
        img[:]=(threshold_3,threshold_2,threshold_1)
    else:
        img[:]=0
    
    cv2.imshow(name, img)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'Palette' in function 'cvGetTrackbarPos'


In [4]:
## 오츠 알고리즘: 히스토그램을 분석하여 자동으로 최적의 임계값을 계산함
# 0부터 255까지 전부 계산하여 평균값과의 차이가 최대화되는 지점을 계산


img=cv2.imread(book_path, cv2.IMREAD_GRAYSCALE)

ret_1, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
ret_2, otsu = cv2.threshold(img, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
print(ret_1, ret_2)    # ret_2를 보면 오츠알고리즘에 따라 계산된 임계값이 출력됨

cv2.imshow('1', img)
cv2.imshow('2', binary)
cv2.imshow('3', otsu)

cv2.waitKey(0)
cv2.destroyAllWindows()

127.0 138.0
